In [66]:
import json
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from workingData import df as wdDF
from sklearn.model_selection import cross_val_score


import requests

# api_key = '7A0JXiyThxo6l9471M7n97tW5LXwXZcx1qePcPYE'



# url = 'https://api.eia.gov/v2/coal/aggregate-production/data/?frequency=annual&data[0]=average-employees&data[1]=labor-hours&data[2]=number-of-mines&data[3]=production&data[4]=productivity&start=2010&end=2020&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=7A0JXiyThxo6l9471M7n97tW5LXwXZcx1qePcPYE'
# response = requests.get(url)
# js = response.json()
# data = pd.json_normalize(js)





with open('natGas.txt', 'r') as f:
    data_str = f.read()

data = json.loads(data_str)

df = pd.DataFrame(data)

df.to_csv('natGas.csv')

df = pd.concat([df, wdDF])




df = df.drop('units', axis=1, inplace=False)
df = df.drop('area-name', axis=1, inplace=False)
df = df.drop('product', axis=1, inplace=False)
df = df.drop('product-name', axis=1, inplace=False)
df = df.drop('process-name', axis=1, inplace=False)
df = df.drop('series-description', axis=1, inplace=False)
df = df.drop('series', axis=1, inplace=False)
df = df.drop('duoarea', axis=1, inplace=False)
df = df.drop('process', axis=1, inplace=False)


lst = []

for i in df['period']:

    lst.append(i)

lst = [*set(lst)]


df['value'] = df['value'].replace(0, None)


for i in lst:


    subset = df[df['period'] == i]
    imputer = SimpleImputer(strategy='median')
    imputer.fit(subset[['value']])
    df['value'] = imputer.transform(df[['value']])





tempDF = pd.read_csv('gdp.csv')

tempCol = tempDF[' GDP ( Billions of US $)']


lst = []

for index, row in tempDF.iterrows():
    period = row.loc['period']
    gdp = row.loc[' GDP ( Billions of US $)']
    vals = (int(period), gdp)
    lst.append(vals)



for i in lst:
    # access the individual elements of the tuple using indexing
    period = i[0]
    gdp = i[1]
    df.loc[df['period'] ==period, 'GDP'] = gdp
    
df = df[df['value'] != 0]
df = df[df['value'].notna()]

# subset_df = df[(df['period'] != 2012) | (df['period'] == 2013)| (df['period'] == 2014)]
# subset_df = df[(df['period'] != 2020) |(df['period'] != 2012) ]




X = df.drop('GDP', axis=1, inplace=False)

y = (df['GDP'])


model = LinearRegression()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

rmse = np.sqrt(mse)
error_percentage = rmse / (y_test.max() - y_test.min()) * 100
print("Percentage Error:", error_percentage)

print(df)


# scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error')
# mse_scores = -scores
# print("Mean Squared Error:", np.mean(mse_scores))




Mean Absolute Error: 462.93066163518927
Percentage Error: 4.792376084448675
      period     value           GDP
0       2021    8994.0  23315.080560
1       2021  122431.0  23315.080560
2       2021   65009.0  23315.080560
3       2021   65009.0  23315.080560
4       2021      72.0  23315.080560
...      ...       ...           ...
3783    2004   65009.0  12217.193198
3784    2004   65009.0  12217.193198
3785    2004     100.0  12217.193198
3786    2004     100.0  12217.193198
3787    2004     100.0  12217.193198

[8788 rows x 3 columns]
